# SageMaker Notebook 

First we check that we are on the right enviroment 

In [1]:
!echo $CONDA_DEFAULT_ENV

automatic


In [2]:
import sys
from IPython.display import clear_output
print(sys.executable)
!conda info --envs

/home/ec2-user/anaconda3/envs/automatic/bin/python
# conda environments:
#
base                     /home/ec2-user/anaconda3
JupyterSystemEnv         /home/ec2-user/anaconda3/envs/JupyterSystemEnv
R                        /home/ec2-user/anaconda3/envs/R
automatic             *  /home/ec2-user/anaconda3/envs/automatic
python3                  /home/ec2-user/anaconda3/envs/python3
pytorch_p310             /home/ec2-user/anaconda3/envs/pytorch_p310
tensorflow2_p310         /home/ec2-user/anaconda3/envs/tensorflow2_p310



In [3]:
!python --version

Python 3.10.11


## Additional Packages for SageMaker

In [4]:
!pip install ipywidgets
%pip install gdown
!pip3 install civitdl
!pip install python-dotenv
clear_output()

In [5]:
import ipywidgets as widgets
from dotenv import load_dotenv
import os

# Load values from .env file
load_dotenv('.env')
# Get the value of HF_TOKEN from .env file
#hf_token = os.getenv('HF_TOKEN')
# Get the value of API_KEY civitai from .env file
API_KEY = os.getenv('API_KEY')

# Installation: Clone webui repository


In [6]:
%cd ~/SageMaker/
!git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui
#create softlink to /tmp/outputs for storing images
#contents of /tmp folder will be deleted when each session ends
!ln -vs /tmp/outputs stable-diffusion-webui/outputs
#create softlink to /tmp in ~/sagemaker-studiolab-notebooks/
!ln -vs /tmp ~/sagemaker-studiolab-notebooks/tmp
#create softlink to /tmp/models in stable-diffusion-webui/models/Stable-diffusion/
!ln -vs /tmp/models stable-diffusion-webui/models/Stable-diffusion/tmp_models
# change default settings (not necessary)
%cd stable-diffusion-webui
!wget https://raw.githubusercontent.com/Miraculix200/StableDiffusionUI_Colab/main/config.json
!wget https://raw.githubusercontent.com/Miraculix200/StableDiffusionUI_Colab/main/ui-config.json
learning_rate = "5e-5:100, 5e-6:1500, 5e-7:2000, 5e-5:2100, 5e-7:3000, 5e-5:3100, 5e-7:4000, 5e-5:4100, 5e-7:5000, 5e-5:5100, 5e-7:6000, 5e-5:6100, 5e-7:7000, 5e-5:7100, 5e-7:8000, 5e-5:8100, 5e-7:9000, 5e-5:9100, 5e-7:10000, 5e-6:10100, 5e-8:11000, 5e-6:11100, 5e-8:12000, 5e-6:12100, 5e-8:13000, 5e-6:13100, 5e-8:14000, 5e-6:14100, 5e-8:15000, 5e-6:15100, 5e-8:16000, 5e-6:16100, 5e-8:17000, 5e-6:17100, 5e-8:18000, 5e-6:18100, 5e-8:19000, 5e-6:19100, 5e-8:20000, 5e-5:20100, 5e-7:21000, 5e-5:21100, 5e-7:22000, 5e-5:22100, 5e-7:23000, 5e-5:23100, 5e-7:24000, 5e-5:24100, 5e-7:25000, 5e-5:25100, 5e-7:26000, 5e-5:26100, 5e-7:27000, 5e-5:27100, 5e-7:28000, 5e-5:28100, 5e-7:29000, 5e-5:29100, 5e-7:30000, 5e-6:30100, 5e-8:31000, 5e-6:31100, 5e-8:32000, 5e-6:32100, 5e-8:33000, 5e-6:33100, 5e-8:34000, 5e-6:34100, 5e-8:35000, 5e-6:35100, 5e-8:36000, 5e-6:36100, 5e-8:37000, 5e-6:37100, 5e-8:38000, 5e-6:38100, 5e-8:39000, 5e-6:39100, 5e-8:40000"
!echo "a photo of a [filewords]" >textual_inversion_templates/hypernetwork2.txt
# install extensions (not necessary)
%cd extensions
!git clone --depth 1 https://github.com/yfszzx/stable-diffusion-webui-images-browser
!git clone --depth 1 https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor.git
clear_output()

## Installation: Download model(s) from Huggingface


Alternatively you can use the JupyterLab file browser to upload one or more .ckpt files
to the ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/ folder

There are optional model downloads below this cell, where a Huggingface token is not necessary
To save disk space, you can move models to /tmp/models and restart the webui. 
These models will be deleted when the runtime ends

# Download model(s)
Check free disk space


In [7]:
!df -h | grep -E 'Avail|home'

Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme2n1     99G   23G   72G  25% /home/ec2-user/SageMaker


# Huggingface  tokens
You only have to enter this once, as the token will persist in ~/.huggingface/token across sessions

In [8]:
import ipywidgets as widgets
from dotenv import load_dotenv
import os

# Load values from .env file
load_dotenv('.env')

# Get the value of HF_TOKEN from .env file
hf_token = os.getenv('HF_TOKEN')

# Create the token_textbox
token_textbox = widgets.Text(
    value=hf_token,
    description='Token:',
)

In [9]:
huggingface_token = hf_token

import os

token_path = os.path.expanduser('~/.huggingface/token')

if os.path.isfile(token_path):
    with open(token_path) as f:
        lines = f.readlines()
    #huggingface_token = lines[0]
    huggingface_token =hf_token
else:
    try:
        token_textbox
    except NameError:
        raise RuntimeError("Enter Huggingface token")
    else:
        !mkdir -p ~/.huggingface
        !echo -n "{token_textbox.value}" > ~/.huggingface/token
        huggingface_token = token_textbox.value

user_header = f"\"Authorization: Bearer {huggingface_token}\""

%cd ~/SageMaker/stable-diffusion-webui/models/Stable-diffusion/

# Choose here which model(s) to download #############################################################################

# Model 1.4
#!wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

# Model 1.5 (8GB) - only necessary for native training, not for embeddings/hypernetworks
#!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt 

# Model 1.5 (4GB)
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt 

# Inpainting model 1.5 (4GB)
#!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt
clear_output()

Recommended Optional: Download variational autoencoder (VAE)
Doing so can improve the quality of generated images, if you select the VAE in the webui settings

Go to these pages and accept the licenses before running the next cell:

https://huggingface.co/stabilityai/sd-vae-ft-ema-original

https://huggingface.co/stabilityai/sd-vae-ft-mse-original

In [10]:
# sd-vae-ft-ema-original
#%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/VAE/
#!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt

# sd-vae-ft-mse-original
#%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/VAE/
#!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt

# sd-vae-ft-mse-original
# Use this if you downloaded v1-5-pruned-emaonly.ckpt above and want to autoselect the VAE every time you load this model in the webui
%cd ~/SageMaker/stable-diffusion-webui/models/Stable-diffusion/
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5-pruned-emaonly.vae.pt 
clear_output()

Optional: Download various models
Change values from False to True to download a model

A Huggingface token is not necessary for these downloads

In [11]:
############################################################

download_to_temp_folder = False # models will be deleted when the SageMaker session ends 

anything_v3 = False # Anything V3 model with VAE
anything_xtron = False # AnythingXtron model
gigachad = False # Gigachad model
midjourney_v4 = True # Midjourney V4 Diffusion
waifu = False # Waifu Division with Anime VAE
modidi = False # Modern Disney Diffusion
nitrodiffusion = False # Multi-Style Model trained from scratch
sd15_cloned = False # Cloned version of model 1.5 (not token necessary)
secret_ai = False # Anime/furry model with VAE and hypernetworks

############################################################

if download_to_temp_folder:
    !mkdir -p /tmp/models
    %cd ~/SageMaker/stable-diffusion-webui/models/Stable-diffusion/tmp_models
else:
    %cd ~/SageMaker/stable-diffusion-webui/models/Stable-diffusion/

if anything_v3:
    !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt
    !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -O Anything-V3.0-pruned.vae.pt

if anything_xtron:
    !wget https://huggingface.co/CrasHthe2nd2/anything-tron/resolve/main/anythingXtron.ckpt
    !wget https://huggingface.co/CrasHthe2nd2/anything-tron/resolve/main/Anything-V3.0.vae.pt -O anythingXtron.vae.pt

if gigachad:
    !wget https://huggingface.co/SpiteAnon/gigachad-diffusion/resolve/main/gigachad_2000.ckpt

if midjourney_v4:
    !wget https://huggingface.co/prompthero/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt

if modidi:
    !wget https://huggingface.co/nitrosocke/mo-di-diffusion/resolve/main/moDi-v1-pruned.ckpt

if nitrodiffusion:
    !wget https://huggingface.co/nitrosocke/Nitro-Diffusion/resolve/main/nitroDiffusion-v1.ckpt

if sd15_cloned:
    !wget https://huggingface.co/acheong08/SD-V1-5-cloned/resolve/main/v1-5-pruned-emaonly.ckpt

if secret_ai:
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/animefull-final-pruned/model.ckpt -O secret_ai.ckpt
    %cd ~/SageMaker/stable-diffusion-webui/models/VAE/
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/animevae.pt
    !mkdir -p ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/hypernetworks/
    %cd ~/SageMaker/stable-diffusion-webui/models/hypernetworks/
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/aini.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime_2.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/anime_3.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_2.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_3.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_kemono.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_protogen.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_scalie.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/furry_transformation.pt
    !wget https://huggingface.co/acheong08/secretAI/resolve/main/stableckpt/modules/modules/pony.pt
# waifu division
if waifu:
    %cd ~/SageMaker/stable-diffusion-webui/models/Stable-diffusion/
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-penultimate-ucg-cont.ckpt

    %cd ~/SageMaker/stable-diffusion-webui/models/VAE/
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt
clear_output()

Sources:

https://huggingface.co/SpiteAnon/gigachad-diffusion

https://huggingface.co/CrasHthe2nd2/anything-tron

https://huggingface.co/Linaqruf/anything-v3.0

https://huggingface.co/prompthero/midjourney-v4-diffusion

https://huggingface.co/hakurei/waifu-diffusion-v1-4

https://huggingface.co/nitrosocke/mo-di-diffusion

https://huggingface.co/nitrosocke/Nitro-Diffusion

https://huggingface.co/acheong08

Installation: Install glib to prevent libgthread error when starting webui
You only need to do this once on your SageMaker SL account

## Civitai Models

In [12]:
!civitdl  "https://civitai.com/models/140737/albedobase-xl" ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion --api-key "$API_KEY"

Now downloading "AlbedoBase XL"...
            - Model ID: 140737
            - Version ID: 329420

Images: 100%|███████████████████████████████| 3.00/3.00 [00:00<00:00, 2.50kiB/s]
Model already existed at the following path:
                - Path: /home/ec2-user/SageMaker/home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL/albedobaseXL_v21-mid_140737-vid_329420.safetensors
Copying to new path...

Download completed for "AlbedoBase XL"
            - Model ID: 140737
            - Version ID: 329420
            - Model Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL/albedobaseXL_v21-mid_140737-vid_329420.safetensors
            - Hashes Directory Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL
            - Metadata Directory Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL/extra_data-vid_329420
            - Images Direc

In [13]:
!civitdl  "https://civitai.com/models/4384/dreamshaper," ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion --api-key "$API_KEY"

Now downloading "DreamShaper"...
            - Model ID: 4384
            - Version ID: 128713

Images: 100%|████████████████████████████████| 3.00/3.00 [00:00<00:00, 38.5iB/s]
Model: 100%|██████████████████████████████| 2.13G/2.13G [01:03<00:00, 33.6MiB/s]

Download completed for "DreamShaper"
            - Model ID: 4384
            - Version ID: 128713
            - Model Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/DreamShaper/dreamshaper_8-mid_4384-vid_128713.safetensors
            - Hashes Directory Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/DreamShaper
            - Metadata Directory Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/DreamShaper/extra_data-vid_128713
            - Images Directory Path: ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/DreamShaper/extra_data-vid_128713
            - Images Prompt/Metadata Directory Path: ./home/ec2-user/Sage

In [14]:
text = '''https://civitai.com/models/140737/albedobase-xl,
https://civitai.com/models/46422/juggernaut,

'''
models = text.split('\n')
with open('batchfile.txt', 'w') as file:
    for model in models:
        file.write(model + '\n')
!civitdl  batchfile.txt  ./home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion --api-key "$API_KEY"

Now downloading "AlbedoBase XL"...
            - Model ID: 140737
            - Version ID: 329420

Images: 100%|████████████████████████████████| 3.00/3.00 [00:00<00:00, 46.5iB/s]
Model already existed at the following path:
                - Path: /home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL/albedobaseXL_v21-mid_140737-vid_329420.safetensors
Copying to new path...
---------
'/home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL/albedobaseXL_v21-mid_140737-vid_329420.safetensors' and './home/ec2-user/SageMaker/stable-diffusion-webui/models/Stable-diffusion/AlbedoBase XL/albedobaseXL_v21-mid_140737-vid_329420.safetensors' are the same file 

---------
Now downloading "Juggernaut"...
            - Model ID: 46422
            - Version ID: 274039

Images: 100%|████████████████

# Installation  Stable Diffusion Webui

In [15]:
#!conda install glib=2.51.0 -y

#Update: Change into Web UI directory and download updates

#This is not strictly necessary. You can run this every time before you start the webui

%cd ~/SageMaker/stable-diffusion-webui
!git pull
#Update: Install webui requirements.txt
#This is necessary sometimes after you did a !git pull above

%cd ~/SageMaker/stable-diffusion-webui
%pip install -r requirements.txt
clear_output()


Note that every time you stop the SageMaker Studio Lab runtime the generated images will be lost, as they are stored in the /tmp folder

In [16]:
##################################

WEBUI_USERNAME="genius"
WEBUI_PASSWORD="artist"

##################################


In [17]:
!pwd

/home/ec2-user/SageMaker/stable-diffusion-webui


In [18]:
# create /tmp/outputs folder for storing generated images for this session
!mkdir -p /tmp/outputs
!mkdir -p /tmp/models

In [19]:
!df -h | grep home

/dev/nvme2n1     99G   40G   55G  42% /home/ec2-user/SageMaker


In [20]:
##################################

WEBUI_USERNAME = "genius"
WEBUI_PASSWORD = "artist"

##################################


In [ ]:
# launch webui
%cd ~/SageMaker/stable-diffusion-webui
ARGS="--disable-console-progressbars --gradio-debug --gradio-auth "+ WEBUI_USERNAME + ":" + WEBUI_PASSWORD+" --share"
!COMMANDLINE_ARGS="$ARGS" REQS_FILE="requirements.txt" python launch.py

/home/ec2-user/SageMaker/stable-diffusion-webui
Python 3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:58:44) [GCC 11.3.0]
Version: v1.7.0
Commit hash: cf2772fab0af5573da775e7437e6acdca424f26e
Cloning Stable Diffusion into /home/ec2-user/SageMaker/stable-diffusion-webui/repositories/stable-diffusion-stability-ai...
Cloning into '/home/ec2-user/SageMaker/stable-diffusion-webui/repositories/stable-diffusion-stability-ai'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 580 (delta 260), reused 229 (delta 229), pack-reused 223
Receiving objects: 100% (580/580), 73.44 MiB | 66.08 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Cloning Stable Diffusion XL into /home/ec2-user/SageMaker/stable-diffusion-webui/repositories/generative-models...
Cloning into '/home/ec2-user/SageMaker/stable-diffusion-webui/repositories/generative-models'...
remote: Enumerating objects: 860, d

Commandline arguments are:
```
--share - create online gradio.app link (currently this doesn't work on SageMaker SL)
--gradio-debug - print outputs to console
--ngrok - start ngrok reverse proxy with supplied auth token
```
End: Commands for after you're done with a session

```bash
COMMANDLINE_ARGS="--disable-console-progressbars --gradio-debug --gradio-auth genius:artist --share"
python launch.py $COMMANDLINE_ARGS
```

Click the square stop button before running these cells
Create .tar.gz archive of images for downloading
Location will be ~/sagemaker-studiolab-notebooks/tmp/outputs.tar.gz and /tmp/outputs.tar.gz

In [ ]:
%cd /tmp
!echo "Creating archive. Please wait..."
!tar -czf /tmp/outputs.tar.gz outputs
!echo "You can now download the archive with generated images"